In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import missingno as msno
import matplotlib.pyplot as plt
from datetime import date
today = date.today()
from tqdm import tqdm
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import sklearn
from wordcloud import WordCloud
import string
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
import pyLDAvis
import pyLDAvis.sklearn
import gensim
from itertools import combinations
from sklearn.tree import DecisionTreeClassifier
from util import *

C:\Users\jerom\anaconda3\envs\PROJET5\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [ ]:
#Lecture du Fichier
#df = pd.read_csv('posts_after_prepocessing.csv', sep = ',', encoding='UTF-8')

In [2]:
#Deserialisation de mon Dataframe traité côté Preprocessing
import pickle
with open('df_afterPreprocessing.pickle', 'rb') as file:
    df = pickle.load(file)

In [3]:
#Préprocessing du champ Tittle et Body
df['TitleBody'] = df['TitleBody'].map(lambda x: BeautifulSoup(x, "html.parser").get_text())
df['TitleBody'] = df['TitleBody'].map(lambda x: x.replace('\n', ' '))
df['TitleBody'] = df['TitleBody'].map(lambda x: x.replace(':', ''))

punct = string.punctuation
for c in punct:
    if c != '#':
        df['TitleBody'] = df['TitleBody'].map(lambda x: x.replace(c, ''))

#Parsing avec NLTK
df['TitleBody_final'] = df.apply(lambda row: nltk.word_tokenize(row['TitleBody'],language='english'), axis=1)

#Supression des StopWord
df['TitleBody_final'] = df.apply(lambda row:removeStopWord(row['TitleBody_final']), axis=1)

#Lemmatisation
#df['TitleBody_final'] = df.apply(lambda row:lemmatisation(row['TitleBody_final']), axis=1)

#Traitement du C#
df['TitleBody_final'] = df.apply(lambda row:processCSharp(row['TitleBody_final']), axis=1)

#Suppressin des numériques
df['TitleBody_final'] = df.apply(lambda row:removeOnlyNumeric(row['TitleBody_final']), axis=1)

In [7]:
df.head(5)

,Id,BODY,Title,Tags,CreationDate,TitleBody,processed_tags_final,processed_body_final,processed_title_final,tags_words,TitleBody_final
0,12051,<p>if i inherit from a base class and want to ...,calling the base constructor in c#,<c#><.net><inheritance><constructor>,2008-08-15 07:39:23,calling the base constructor in c# if i inheri...,"[c#, .net]","[inherit, base, class, want, pass, something, ...","[calling, base, constructor, c#]","[c#, .net, inheritance, constructor]","[calling, base, constructor, c#, inherit, base..."
1,17319422,<p>i am using pycharm on windows and want to c...,how do i set the maximum line length in pycharm?,<python><pycharm><pep8>,2013-06-26 12:00:31,how do i set the maximum line length in pychar...,[python],"[using, pycharm, windows, want, change, settin...","[set, maximum, line, length, pycharm]","[python, pycharm, pep8]","[set, maximum, line, length, pycharm, using, p..."
2,32664,<p>can anyone tell me if there is a way with g...,is there a constraint that restricts my generi...,<c#><generics><constraints>,2008-08-28 16:04:49,is there a constraint that restricts my generi...,"[c#, generics]","[anyone, tell, way, generics, limit, generic, ...","[constraint, restricts, generic, method, numer...","[c#, generics, constraints]","[constraint, restricts, generic, method, numer..."
3,8763125,<p>i would like to get the keys of a javascrip...,get array of object's keys,<javascript><ecmascript-5>,2012-01-06 19:12:38,get array of objects keys i would like to get ...,[javascript],"[would, like, get, keys, javascript, object, a...","[get, array, objects, keys]","[javascript, ecmascript-5]","[get, array, objects, keys, would, like, get, ..."
5,33923,"<p>whilst starting to learn lisp, i've come ac...",what is tail recursion?,<algorithm><language-agnostic><functional-prog...,2008-08-29 03:48:03,what is tail recursion whilst starting to lear...,"[algorithm, language-agnostic]","[whilst, starting, learn, lisp, ive, come, acr...","[tail, recursion]","[algorithm, language-agnostic, functional-prog...","[tail, recursion, whilst, starting, learn, lis..."


In [4]:
#Utilisation de la librairie MultilabelBinarize pour créer un matrice de Tags
from sklearn.preprocessing import MultiLabelBinarizer

tags = df["processed_tags_final"]

mlb = MultiLabelBinarizer()
tags_mlb = mlb.fit_transform(df["processed_tags_final"].to_numpy())

## Création du jeu d'entrainement et du jeu de test 

In [5]:
from sklearn.model_selection import train_test_split

X = df['TitleBody_final']
y = tags_mlb

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=.25)

## Vectorisation avec TF-IDF

In [6]:
vectorizer = TfidfVectorizer(tokenizer=dummy,preprocessor=dummy,
                            max_features=12000)
vectorizer.fit(X_train)
X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)

C:\Users\jerom\anaconda3\envs\PROJET5\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
#Serialisation de la vectorisation
pickle.dump(vectorizer, open('vectorizer.pickle', 'wb')) #Saving DF

In [33]:
import joblib
#Serialisation avec JOBLIB
joblib.dump(vectorizer,"vectorizer.pickle")

['vectorizer.pickle']

In [34]:
X_train_vec.shape

(11378, 12000)

## Modèle Naive Bayse après TF_IDF
* Utilisation de la méthode OneVsRest qui calcule toutes les étiquettes de sortie (Attention aux performances car n modèles suivant les tags de sortie)
* J'utilise les données après un TF_IDF

In [7]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB

nb_clf = OneVsRestClassifier(MultinomialNB(fit_prior=True, 
                                           class_prior=None))
nb_clf.fit(X_train_vec, y_train)

OneVsRestClassifier(estimator=MultinomialNB())

In [8]:
y_pred_nb = nb_clf.predict(X_test_vec)
print_score(y_test, y_pred_nb)

Hamming loss : 0.01577643026627999
Subset Accuracy :  0.04323754284207751
F1-score :  0.08445532435740516
Jaccard :  0.044089456869009586


## Modèle Arbre de décision après NMF (Comme réduction de dimension)

In [9]:
#D'abord réalisation d'une NMF avec le paramètre optimum calculté précédemment (8 Topics)
from sklearn.decomposition import NMF
k = 8
# create the model, specifiying the initialization strategy and the number of topics to produce
model = NMF(n_components=k,init="nndsvd",random_state=42,max_iter = 300) 
# apply the model and extract the two factor matrices
W_train = model.fit_transform(X_train_vec)
W_test = model.fit_transform(X_test_vec)

In [10]:
#Arbre de décision
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(W_train, y_train)

DecisionTreeClassifier(random_state=0)

In [11]:
y_pred_nb = clf.predict(W_test)
print_score(y_test, y_pred_nb)

Hamming loss : 0.030925388874242024
Subset Accuracy :  0.04007382019509623
F1-score :  0.11257376305038584
Jaccard :  0.059644059644059645


## Modèle Arbre de décision sans NMF

In [12]:
#Arbre de décision
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train_vec, y_train)

DecisionTreeClassifier(random_state=42)

In [13]:
y_pred_nb = clf.predict(X_test_vec)
print_score(y_test, y_pred_nb)

Hamming loss : 0.013548642235697338
Subset Accuracy :  0.30951753229633533
F1-score :  0.5738452608010615
Jaccard :  0.40237236888010236


In [21]:
#Serialisation du modele
pickle.dump(clf, open('decisionTreeClassifier.pickle', 'wb')) #Saving DF

In [37]:
#Serialisation avec JOBLIB
joblib.dump(clf,"decisionTreeClassifier.pickle")

['decisionTreeClassifier.pickle']

## Modèle Arbre de décision après TruncateSVD
* Contrairement à l'ACP TruncateSVD supporte les matrices creuses

In [14]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=1000, n_iter=7, random_state=42)
svd.fit(X_train_vec)
X_train_SVD = svd.transform(X_train_vec)
X_test_SVD = svd.transform(X_test_vec)

#Arbre de décision
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train_SVD, y_train)

DecisionTreeClassifier(random_state=42)

In [15]:
y_pred_nb = clf.predict(X_test_SVD)
print_score(y_test, y_pred_nb)

Hamming loss : 0.021523859741629316
Subset Accuracy :  0.16530450830477195
F1-score :  0.35031036129237625
Jaccard :  0.21234925229136517


In [ ]:
X_train_vec.shape

## Modèle Arbre de décision après ACP

In [16]:
from sklearn import decomposition
# choix du nombre de composantes à calculer
n_comp = 50

features = df.columns

# Calcul des composantes principales
pca = decomposition.PCA(n_components=n_comp,random_state=42)
pca.fit(X_train_vec.toarray())
X_train_PCA = pca.transform(X_train_vec.toarray())
X_test_PCA = pca.transform(X_test_vec.toarray())

#Arbre de décision
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train_PCA, y_train)

DecisionTreeClassifier(random_state=42)

In [17]:
y_pred_nb = clf.predict(X_test_PCA)
print_score(y_test, y_pred_nb)

Hamming loss : 0.02165568151858687
Subset Accuracy :  0.16293171631953599
F1-score :  0.3483021263091083
Jaccard :  0.2108752041502546


In [ ]:
mlb.classes_

In [21]:
#Fonction qui compte les bonnes prédictions et les mauvaises prédictions par tags
def scoreTags():
    resultat = []
    for index, word in enumerate(mlb.classes_):
        GoodPrediction = 0
        BadPrediction = 0
        for i in range(0,len(y_test)):
            if (y_test[i][index] == 1):
                if (y_test[i][index] == y_pred_nb[i][index]):
                    GoodPrediction += 1
                else:
                    BadPrediction +=1
        resultat.append((word,GoodPrediction,BadPrediction))
    return resultat

In [22]:
comptagePrediction = scoreTags()

In [ ]:
for res in y_test:

In [ ]:
len(y_test)

In [23]:
comptagePrediction

[('.net', 12, 121),
 ('ajax', 3, 20),
 ('algorithm', 2, 29),
 ('android', 120, 114),
 ('android-studio', 10, 8),
 ('angular', 8, 38),
 ('angularjs', 1, 34),
 ('arrays', 43, 54),
 ('asp.net', 3, 26),
 ('asp.net-mvc', 2, 31),
 ('bash', 46, 75),
 ('branch', 5, 22),
 ('c', 10, 49),
 ('c#', 78, 190),
 ('c++', 33, 96),
 ('c++11', 1, 17),
 ('class', 0, 19),
 ('cocoa-touch', 0, 20),
 ('collections', 1, 23),
 ('command-line', 7, 33),
 ('css', 102, 66),
 ('database', 1, 30),
 ('dataframe', 7, 26),
 ('date', 12, 30),
 ('datetime', 8, 29),
 ('debugging', 0, 20),
 ('dictionary', 7, 30),
 ('django', 0, 23),
 ('docker', 19, 13),
 ('dom', 3, 17),
 ('eclipse', 5, 26),
 ('exception', 2, 25),
 ('file', 3, 26),
 ('forms', 1, 16),
 ('function', 2, 26),
 ('generics', 1, 15),
 ('git', 234, 25),
 ('github', 12, 44),
 ('go', 0, 17),
 ('google-chrome', 1, 24),
 ('html', 71, 135),
 ('http', 2, 32),
 ('intellij-idea', 1, 17),
 ('ios', 21, 79),
 ('iphone', 2, 23),
 ('java', 178, 178),
 ('java-8', 0, 15),
 ('javasc